In [0]:
import pandas as pd
from google.colab import files 
import numpy as np
import matplotlib.pyplot as plt

In [0]:
uploaded_training = files.upload()


Saving train.csv to train (9).csv


In [0]:
uploaded_test = files.upload()


Saving sample.csv to sample (9).csv


In [0]:
for fn1 in uploaded_training.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn1, length=len(uploaded_training[fn1])))
  
for fn2 in uploaded_test.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn2, length=len(uploaded_test[fn2])))

User uploaded file "train.csv" with length 17327 bytes
User uploaded file "sample.csv" with length 6279 bytes


In [0]:
import io 
train_df=pd.read_csv(io.StringIO(uploaded_training['train.csv'].decode('cp1252')),index_col=0)
test_df=pd.read_csv(io.StringIO(uploaded_test['sample.csv'].decode('cp1252')),index_col=0)

In [0]:
train_df.head(7)

In [0]:
test_df.head(7)

In [0]:
Y_Train_df=train_df.sort_index(ascending=True)
Y_Test_df=test_df.sort_index(ascending=True)


In [0]:
Y_Train=Y_Train_df.values
Y_Test=Y_Test_df.values


In [0]:
!git clone https://github.com/adilahiri/7Challenge.git

fatal: destination path '7Challenge' already exists and is not an empty directory.


In [0]:
! ls */Train/*

In [0]:
! ls */Test/*

In [0]:
# Run it to obtain reproducible results across machines (from keras.io)
from __future__ import print_function
import tensorflow as tf
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [0]:
import glob
from keras.preprocessing import image as kImage

In [0]:
def getData(dataset_dir,Y_list):
    X_list= sorted(glob.glob(os.path.join(dataset_dir,'*.jpg')))
    #Y_list = sorted(glob.glob(os.path.join(dataset_dir, 'y' ,'*.png')))
   
    X= []
    Y= []
    for i in range(len(X_list)):
        # Load input image
        x = kImage.load_img(X_list[i])
        x = kImage.img_to_array(x)
        X.append(x)
        
        # Load ground-truth label and encode it to label 0 and 1
        x = Y_list[i]
        #x = kImage.img_to_array(x)
        #x /= 255.0
        #x = np.floor(x)
        Y.append(x)
    X = np.asarray(X)
    Y = np.asarray(Y)
    
    # Shuffle the training data
    #idx = list(range(X.shape[0]))
    #np.random.shuffle(idx)
    #X = X[idx]
    #Y = Y[idx]
    return X, Y

In [0]:
dataset_path = os.path.join('7Challenge', 'Train')
Y_List=Y_Train.tolist()
X_Train_CNN,Y_Train_CNN = getData(dataset_path,Y_List)



In [0]:
X_Train_CNN.shape

(792, 300, 400, 3)

In [0]:
#X_Train_CNN=X_Train_CNN/255
mean = X_Train_CNN.mean(axis=0)
std = X_Train_CNN.std(axis=0)
X_Train_CNN = (X_Train_CNN - mean) / std



In [0]:
dataset_dir = os.path.join('7Challenge', 'Test')
X_list= sorted(glob.glob(os.path.join(dataset_dir,'*.jpg')))
X_list
Y_list=Y_Test.tolist()


In [0]:
X= []
Y= []
for i in range(len(X_list)):
  # Load input image
  x = kImage.load_img(X_list[i])
  x = kImage.img_to_array(x)
  X.append(x)
        
  # Load ground-truth label and encode it to label 0 and 1
  x = Y_list[i]
  #x = kImage.img_to_array(x)
  #x /= 255.0
  #x = np.floor(x)
  Y.append(x)


In [0]:
X[44]=X[44].reshape(300,400,3)
X[241]=X[241].reshape(300,400,3)
X[312]=X[312].reshape(300,400,3)
X[355]=X[355].reshape(300,400,3)
X[454]=X[454].reshape(300,400,3)
X1 = np.asarray(X)


In [0]:
#X1=X1/255
X1 = (X1 - mean) / std

In [0]:
X_Train_CNN

In [0]:
X1

In [0]:
#from sklearn.model_selection import train_test_split
#x_train, x_validation, y_train, y_validation = train_test_split(X_Train_CNN, Y_Train_CNN, test_size = 0.10, random_state = 0)

In [0]:
from sklearn.metrics import confusion_matrix
import itertools
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor


In [0]:
#model = Sequential()
#model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal',input_shape=(300,400,3)))
#model.add(MaxPool2D((2, 2)))
#model.add(Dropout(0.20))#0.20
#model.add(Conv2D(64, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal'))
#model.add(MaxPool2D((2, 2)))
#model.add(Dropout(0.25))#0.20
#model.add(Flatten())
#model.add(Dense(256, activation='linear'))
#model.add(BatchNormalization())
#model.add(Dropout(0.25))#0.25
#model.add(Dense(1, activation='linear'))



In [0]:
#best case all linear 
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal',input_shape=(300,400,3)))
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D((2, 2)))
#model.add(Dropout(0.20))#0.20
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2, 2)))
#model.add(Dropout(0.25)) #0.25
model.add(Conv2D(128, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal')) #best is linear
#model.add(Conv2D(128, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal')) #linear
#model.add(MaxPool2D(pool_size=(2, 2)))
#model.add(Conv2D(256, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))#linear
#model.add(Conv2D(256, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))#linear
#model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))#0.25
model.add(Flatten())
#model.add(Dense(256, activation='linear'))
model.add(Dense(128, activation='linear'))
model.add(Dense(64, activation='linear'))
model.add(BatchNormalization())
#model.add(Dropout(0.25))#0.25
model.add(Dense(1, activation='linear'))


In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal',input_shape=(300,400,3)))
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D((2, 2)))
#model.add(Dropout(0.20))#0.20
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2, 2)))
#model.add(Dropout(0.25)) #0.25
model.add(Conv2D(128, (3, 3), activation='linear',padding='same',kernel_initializer='he_normal'))
model.add(Dropout(0.25))#0.25
model.add(Flatten())
model.add(Dense(128, activation='linear'))
model.add(Dense(64, activation='linear'))
model.add(BatchNormalization())
#model.add(Dropout(0.25))#0.25
model.add(Dense(1, activation='linear'))

In [0]:
optimizer1 = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='mse', optimizer=optimizer1, metrics=['mse'])


In [0]:
batch_size=5
epochs=100

In [0]:
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False, 
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1, 
        horizontal_flip=False,  
        vertical_flip=False) 

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(X_Train_CNN, Y_Train_CNN, test_size = 0.20, random_state = 42)

In [0]:
datagen.fit(x_train)

In [0]:
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_validation,y_validation), steps_per_epoch=x_train.shape[0] // batch_size)

126/126 [==============================] - 36s 289ms/step - loss: 0.0640 - mean_squared_error: 0.0640 - val_loss: 0.0642 - val_mean_squared_error: 0.0642
Epoch 65/100
126/126 [==============================] - 36s 289ms/step - loss: 0.0622 - mean_squared_error: 0.0622 - val_loss: 0.0809 - val_mean_squared_error: 0.0809
Epoch 66/100
126/126 [==============================] - 36s 289ms/step - loss: 0.0621 - mean_squared_error: 0.0621 - val_loss: 0.1039 - val_mean_squared_error: 0.1039
Epoch 67/100
126/126 [==============================] - 36s 289ms/step - loss: 0.0558 - mean_squared_error: 0.0558 - val_loss: 0.0589 - val_mean_squared_error: 0.0589
Epoch 68/100
126/126 [==============================] - 36s 289ms/step - loss: 0.0612 - mean_squared_error: 0.0612 - val_loss: 0.0634 - val_mean_squared_error: 0.0634
Epoch 69/100
126/126 [==============================] - 36s 289ms/step - loss: 0.0613 - mean_squared_error: 0.0613 - val_loss: 0.0940 - val_mean_squared_error: 0.0940
Epoch 70/10

In [0]:
history = model.fit(X_Train_CNN, Y_Train_CNN, epochs=10, batch_size=5, validation_split=0.20)#batch 5 best 

In [0]:
import statistics 
statistics.mean(history.history['val_mean_squared_error']) #datagen gives 0.31 || normalized linear gives 0.635 ||standardized=0.106|| standardized and batch 16 0.081||best 0.1741-0.09||
#|| best + 1 extra 128 layer using standardization gives 0.115 ||best + 1 extra 128 layer + 2 extra 256 layers using standardization gives 0.0945|| + extra 256 in fully connected 0.1315 standardized
#


0.09070363052816638

In [0]:
history.history['val_mean_squared_error']

[0.07589704158595828,
 0.30175676745063856,
 0.10073875134471078,
 0.25152741053944117,
 0.06434538067792947,
 0.1174815011169738,
 0.06263047865586086,
 0.06318621046302265,
 0.08380671875527913,
 0.07129041025262771,
 0.34433578128147424,
 0.07352953644340518,
 0.06935865307847659,
 0.07849320250639469,
 0.09051578594723118,
 0.08209346137566012,
 0.08937375849030865,
 0.07771425186466051,
 0.0613563470278849,
 0.11313265943958324,
 0.07949609669590217,
 0.0818512185752017,
 0.06534013065140483,
 0.06348173625242996,
 0.10577693408202825,
 0.06496706036499368,
 0.06353025710936519,
 0.0941609007089674,
 0.10152850448558072,
 0.09270244771204654,
 0.2502977577207973,
 0.183973102918211,
 0.08206657801005028,
 0.06614120462733619,
 0.1819719466023475,
 0.06262893560371778,
 0.12215208430897515,
 0.08614843580051788,
 0.09622049364759205,
 0.06348374575587376,
 0.07381900956959657,
 0.07663804286821459,
 0.06683328773814647,
 0.10712500998117733,
 0.06601514123854632,
 0.086676683739624

In [0]:
#X1=X1/255
X1

In [0]:
pred = model.predict(X1)

In [0]:
pred

In [0]:
Y_Test_df['DGCI']=pred

In [0]:
Y_Test_df.head(3)

,DGCI
Id,
223518,0.418749
225310,0.367834
384141,0.417414


In [0]:
test2=test_df


In [0]:
test2=test2.reset_index(level=0, inplace=False)
test2.head(3)

,Id,DGCI
0,79783930,0
1,40239934,0
2,50410534,0


In [0]:
a=np.argwhere(test2['Id']==40239934)
a[0,0]

1

In [0]:
for iter in range(528):
  index_Y_Test_df=Y_Test_df.index.values[iter]
  location=np.argwhere(test2['Id']==index_Y_Test_df)[0,0]
  test2['DGCI'].iloc[location]=Y_Test_df.DGCI.values[iter]
  

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
test2.head(10)

,Id,DGCI
0,79783930,0.426492
1,40239934,0.449278
2,50410534,0.405231
3,47165338,0.402077
4,16455157,0.424567
5,90938592,0.430090
6,1317745,0.445941
7,59791582,0.367885
8,14525631,0.451364
9,88602343,0.366732


In [0]:
np.argwhere(Y_Test_df.index.values==50410534)

array([[277]])

In [0]:
Y_Test_df.iloc[277,:]

DGCI    0.405231
Name: 50410534, dtype: float32

In [0]:
test2.to_csv('sample_submit.csv',sep=',')

In [0]:
files.download('sample_submit.csv')